In [ ]:

from semanticscholar import SemanticScholar
# import pandas as pd
import glob
import requests
import urllib.request
import json
# import csv
import os
import shutil
# import itertools
import sys
# import time
#from datetime import datetime
from tqdm import tqdm
import time

# Global Settings

In [ ]:
#### Initial Parameters 
#### Global Vars & Flags

# replace with query results length and parse length using modulo

offset = 0
limit = 1
query = "WNT+signalling"
increment = 1
NumResults = 100
s2_api_key = 'qZWKkOKyzP5g9fgjyMmBt1MN2NTC6aT61UklAiyw'

#### path variables 
# data_main = True
# if data_main:
#     root_path = "E:\Data\Kaggle-Covid\document_parses" # Graham's 
# else:
#     root_path = "C:\My files\Courses\CIS6050\Data\document_parses" # Abdullah's

# paths_csv = True

# # temp vars 
# source_dir1 = "E:\Data\Kaggle-Covid\document_parses\pdf_json"
# source_dir2 = "E:\Data\Kaggle-Covid\document_parses\pmc_json"
# dest_dir = "E:\Data\Kaggle-Covid\document_parses\\abstracts_json"


# Utilities

In [ ]:
#### UTILS

# Create json file
def write_to_json(file_in_any, file_out_json):
    with open(file_out_json + ".json", "w") as outfile:
        json.dump(file_in_any, outfile)
        
# Copy files from one directory to another
def copy_file_to_dir(source, destinination):
    for filename in glob.glob(os.path.join(source_dir2, '*.*')):
        shutil.copy(filename, dest_dir) 



#### Query Fields:
- offset: enumerate start
- limit: enumerate end (max 100)
- paperId - Always included
- authorId - Always included
- name - Always included
- title - Included if no fields are specified
- authors - Up to 500 will be returned
- externalIds
- url
- abstract
- venue
- year
- referenceCount
- citationCount
- influentialCitationCount
- isOpenAccess
- fieldsOfStudy
- s2FieldsOfStudy


In [ ]:
# modulo the starting and ending query numbers to arrive at 100, 1000, etc increments
def get_data(query=str, offset=int, limit=int, increment=int, NumResults=int):
    '''
    query: the string to be searched on Semantic Search
    offset: The starting result to retrieve
    limit: The final result to retrieve
    '''
    
    while limit <= NumResults:
        url = f"https://api.semanticscholar.org/graph/v1/paper/search?query={query}&fields=abstract,authors,fieldsOfStudy&offset={offset}&limit={limit}"
        with urllib.request.urlopen(url) as webaddress:
            data = json.loads(webaddress.read().decode())
            #print(data)
        
        offset += increment
        limit += increment
    
    return data 

In [ ]:
data = get_data(query, offset, limit, increment, NumResults)

In [ ]:
offset = 10000
limit = 100
query = "WNT+signalling"
increment = 100
NumResults = 20000
s2_api_key = 'qZWKkOKyzP5g9fgjyMmBt1MN2NTC6aT61UklAiyw'

data = []

while offset <= NumResults:
    url = f"https://api.semanticscholar.org/graph/v1/paper/search?query={query}&fields=abstract,authors,fieldsOfStudy&offset={offset}&limit={limit}"

    try:
        r = requests.get(url,timeout=3)
        r.raise_for_status()
        with urllib.request.urlopen(url) as webaddress:
            response = json.loads(webaddress.read().decode())
            
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
        print(offset)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
        print(offset)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
        print(offset)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
        print(offset)
        
    data.extend(response["data"])
    offset += increment   
        

In [ ]:
print(type(data))
print(len(data))
#print(data)
write_to_json(data, "authorIDs_test")

# Main

In [ ]:
# Parse the dataset from Requests for unique AuthorIDs
## extracting authorIDs from paper.json, storing in a set

#data = get_data(query, offset, limit, increment, NumResults)

authorIDs = []
with open("authorIDs_test.json", 'r', encoding='utf-8') as papers:
    papers_data = json.load(papers)
    #print(type(papers_data) 
    for idx, paper in tqdm(enumerate(papers_data), total=len(papers_data)):        
        try:
            for author in paper["authors"]:
                try:
                    if author["authorId"]:
                        authorIDs.append(author["authorId"])
                    else: 
                        continue
                except:
                    #print the error message from sys
                    print(sys.exc_info()[0])  
        except:   
            #print the error message from sys
            print(sys.exc_info()[0])
            
print(len(authorIDs))
aID = set(authorIDs)
authorIDs_final = list(aID)
print(len(authorIDs_final))

In [ ]:
print(authorIDs_final)

In [ ]:
# 100 req/sec transfer rate.

t = time.process_time()
sch = SemanticScholar(api_key=s2_api_key)
count = 0
authors_abstracts = []
for authorID in authorIDs_final[:1000]:
    if not (count / 100).is_integer():
        authors_abstracts.append(sch.author(authorID))
        count += 1
        print("test2")
    else:
        authors_abstracts.append(sch.author(authorID))
        count += 1
        print("test2")
write_to_json(authors_abstracts, "authors_abstracts")

In [ ]:
# 100 req/sec transfer rate.

t = time.process_time()
sch = SemanticScholar(api_key=s2_api_key)
authors_abstracts = []
i = 0
for i in range(0, len(authorIDs_final[:2000])):
    try:
        if not (i / 100).is_integer(): 
            authors_abstracts.append(sch.author(authorIDs_final[i]))
            #time.sleep(3.1)
            i += 1
        else: 
            authors_abstracts.append(sch.author(authorIDs_final[i]))
            print(f'author info retrieved: {i}')
            #time.sleep(3.1)
            i += 1
    
    except:
        # print the error message from sys
        print(sys.exc_info()[0], "continue")
        print(i)
        i -= 1
        #print(sys.exc_info()[0])


write_to_json(authors_abstracts, "authors_abstracts")
duration = time.process_time() - t
print(f"completed after {duration} seconds")   
        

In [ ]:
print(len(authors_abstracts))
#print(type(authors_abstracts))
#print(authors_abstracts[70:74])
#write_to_json(authors_abstracts[70:74], "authors_abstracts_test")

In [ ]:
sch = SemanticScholar(api_key=s2_api_key)
count = 0
authors_abstracts = []
for authorID in authorIDs_final[:100]:
    try:

        if (t % 5) == 0:
            authors_abstracts.append(sch.author(authorID))
            #time.sleep(3.1)
            
            write_to_json(authors_abstracts, "authors_abstracts") 
            print(f"wrote author info to json at {t} minute")
            
            count+=1
            print(f'author info retrieved: {count}')
        
        else:
            authors_abstracts.append(sch.author(authorID))
            #time.sleep(3.1)
            count+=1
            print(f'author info retrieved: {count}')
            
    except:
        # print the error message from sys
        print(sys.exc_info()[0], "continue")
        print(sys.exc_info()[0]) 
        


In [ ]:
#### Wrap into a query function with input: data

# use the authorID_list to crawl semantic scholar for all papers by a specific author

sch = SemanticScholar(api_key=s2_api_key)
def SS_query(sch, data=list):
    '''
    Hits the Semantic Scholar API and returns all abstracts per author 
    sch: The semantic scholar library object
    data: The result from Requests after parsed for authorIDs
    '''
    
    count = 0
    authors_abstracts = []
    for authorID in data[:100]:
        try:
            t = datetime.now().minute
            if (t % 5) == 0:
                authors_abstracts.append(sch.author(authorID))
                #time.sleep(3.1)
                
                write_to_json(authors_abstracts, "authors_abstracts") 
                print(f"wrote author info to json at {t} minute")
                
                count+=1
                print(f'author info retrieved: {count}')
            
            else:
                authors_abstracts.append(sch.author(authorID))
                #time.sleep(3.1)
                count+=1
                print(f'author info retrieved: {count}')
                
        except:
            # print the error message from sys
            print(sys.exc_info()[0], "continue")
            print(sys.exc_info()[0]) 
            
    write_to_json(authors_abstracts, "authors_abstracts")
    print(f"wrote author info to json at {t} minute")  


In [ ]:
# #### DO NOT RUN UNLESS SURE ####

# ## loading cleaned papers data to pandas
# papers_path = "E:\Data\Kaggle-Covid\clean_df.csv"
# papers_df = pd.read_csv(papers_path)
# #papers_df.head()


In [ ]:
# #### DO NOT RUN UNLESS SURE ####

# ## hitting the semanticScholar API using the paper IDs from papers_df to retrieve authorID
# ## store AuthorID in a json 
# sch = SemanticScholar(timeout=2)
# paperIDs_list = papers_df['paper_id'].tolist()
# authorID = []

# count = 0
# papers = []
# for paperID in paperIDs_list[:12500]:
#     try:
#         t = datetime.now().minute
#         if (t % 5) == 0:
#             papers.append(sch.paper(paperID))
#             time.sleep(3.1)
            
#             write_to_json(papers, "papers") 
#             print(f"wrote papers to json at {t} minute")
            
#             count+=1
#             print(f'papers retrieved: {count}')
        
#         else:
#             papers.append(sch.paper(paperID))
#             time.sleep(3.1)
#             count+=1
#             print(f'papers retrieved: {count}')
            
#     except:
#         # print the error message from sys
#         print(sys.exc_info()[0])
#         time.sleep(330)   
#         continue
    
# write_to_json(papers, "papers")  

In [ ]:
#### NOT RUN UNLESS NECESSARY ####
# creating the unique list of authors

# #parse the data from get_data to retrive the AuthorIDs
# #sort and dedupe the list
# authorID_list = data['author_id'].tolist()

# json_dir = glob.glob(f'{root_path}/**/*.json', recursive=True)
# authors_list = []
# for file in json_dir:
#     with open(file, 'r', encoding='utf-8') as f:
#         data = json.load(f)
#         for item in data["metadata"]["authors"]:
#             if item not in authors_list:
#                 authors_list.append(item)
                
# # print(len(authors_list))


In [ ]:
# ## hitting the semanticScholar API using the author IDs from authorIDs_list to retrieve all author info + abstracts
# ## store author info in a json 

# sch = SemanticScholar(timeout=2)
# authorIDs_list = woduplicates = list(authorIDs_final)

# count = 0
# authors_abstracts = []
# for authorID in authorIDs_list[:12500]:
#     try:
#         t = datetime.now().minute
#         if (t % 5) == 0:
#             authors_abstracts.append(sch.author(authorID))
#             time.sleep(3.1)
            
#             write_to_json(authors_abstracts, "authors_abstracts") 
#             print(f"wrote author info to json at {t} minute")
            
#             count+=1
#             print(f'author info retrieved: {count}')
        
#         else:
#             authors_abstracts.append(sch.author(authorID))
#             time.sleep(3.1)
#             count+=1
#             print(f'author info retrieved: {count}')
            
#     except:
#         # print the error message from sys
#         print(sys.exc_info()[0], "continue")
#         print(sys.exc_info()[0]) 
         
#         # if isinstance(sys.exc_info()[0], type):
#         #     print(sys.exc_info()[0], "continue") 
#         #     continue
#         # else:
#         #     print(sys.exc_info()[0]) 
#         #     time.sleep(330)  
    
# write_to_json(authors_abstracts, "authors_abstracts")
# print(f"wrote author info to json at {t} minute")  

In [ ]:
# #### NOT RUN UNLESS NECESSARY #### 
# # WIP: authors are matched and text + id is added to json based on the match. Takes forever. Still WIP

# id = 0
# with open("Paths_JSON_clean.csv", "r") as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=",")
    
#     # itertools.islice let's us take only a couple items from csv_object item
#     for row in itertools.islice(csv_reader, 20):
        
#         # there will be missing look ups
#         try:
            
#             # flattens the iterator list item to string
#             filename = (''.join(row))
#             abstract_dir = str(root_path) + "/abstracts_json/" + filename  
#             print(abstract_dir)
            
#             # open up the abstracts json using the filename 
#             with open(abstract_dir, 'r', encoding='utf-8') as abstract:
#                 abstract_data = json.load(abstract)
                
#                 # checks if an abstract exists
#                 for text in abstract_data["abstract"]:
#                     if text and abstract_data["metadata"]["authors"]:
#                         for author in abstract_data["metadata"]["authors"]:
#                             #print(len(author))
#                             with open("authors_test.json", 'r', encoding='utf-8') as authors:
#                                 authors_data = json.load(authors)
#                                 if author in authors_data:
#                                     id += 1
#                                     loc = authors_data.index(author)  
#                                     print(text)
#                                     print(id)   
#                                     print(loc)
                                    
#                                     #print(text)
#                                 #print(author)
#                                 #print(type(authors_data))
#                         print(type(text))
#                     else:
#                         continue       
#         except:
            
#             # print the error message from sys
#             print(sys.exc_info()[0])
            
#             continue    
